In [1]:
import numpy as np
import pandas as pd
import requests
from urllib3 import Timeout, PoolManager
from urllib3 import PoolManager
import os
import sys

In [2]:
sys.path.append(os.path.abspath('../Helper Functions'))
from objectives import *

In [3]:
'''
Collecting games data from all three servers at once EUW, NA, and KR as the ratelimit is defined per region so we can speed up our process of collecting games
by introducing all regions of interest at once
'''

'\nCollecting games data from all three servers at once EUW, NA, and KR as the ratelimit is defined per region so we can speed up our process of collecting games\nby introducing all regions of interest at once\n'

In [4]:
timeout = Timeout(connect=10.0, read=15.0)      # Set up timeout to avoid stucking in the request and forcing to restart the kernel
http = PoolManager(timeout=timeout)

In [5]:
dev_key = '' # Enter your roit dev key

In [6]:
def server_match_data(games,match, matches_id , match_data, timeline_match):    # General function to call per region 
    players_counter = len(match_data['info']['participants'])
    if match not in matches_id and players_counter == 10 and match_data['info']['gameMode'] == 'CLASSIC':
        first_tower = False
        games_dic, players_id, flag = championName_playerId(match_data)  # Define games_dic and players_id as two dictionaries for that game
        if flag == False:
            return False
        red_towers, blue_towers ,red_top_inhib , red_mid_inhib ,red_bot_inhib, blue_top_inhib, blue_mid_inhib ,blue_bot_inhib,red_fire_dragons,red_water_dragons,red_earth_dragons,red_air_dragons,red_hextech_dragons,red_chemtech_dragons,red_elder_dragons,blue_fire_dragons,blue_water_dragons,blue_earth_dragons,blue_air_dragons,blue_hextech_dragons,blue_chemtech_dragons,blue_elder_dragons,red_barons,blue_barons,red_rift,blue_rift,red_hordes,blue_hordes, kills, deaths, assists, kda, red_first_tower, red_first_blood, blue_first_tower, blue_first_blood = intialization(players_id)   # Intializations
        for i in range(len(timeline_match['info']['frames'])):
            if i == 0:
                continue
            for j in range(len(timeline_match['info']['frames'][i]['events'])):
                event_type = timeline_match['info']['frames'][i]['events'][j]['type']
                participant_frame = timeline_match['info']['frames'][i]['participantFrames']
                # Buildings, which are turrets and inhibs
                if  event_type == 'BUILDING_KILL':
                    # Turrets
                    if timeline_match['info']['frames'][i]['events'][j]['buildingType'] == "TOWER_BUILDING":
                        lane_type = timeline_match['info']['frames'][i]['events'][j]['laneType']
                        tower_type = timeline_match['info']['frames'][i]['events'][j]['towerType']
                        tower = lane_type + " " +tower_type  
                        if first_tower == False:
                            first_tower = True
                        else:
                            first_tower = False

                        if timeline_match['info']['frames'][i]['events'][j]['teamId'] == 200: # red team tower destroyed
                            red_towers.remove('RED ' + tower)
                            if first_tower == True:
                                blue_first_tower = True

                        
                        elif timeline_match['info']['frames'][i]['events'][j]['teamId'] == 100: # blue team tower destroyed
                            blue_towers.remove('BLUE ' + tower)
                            if first_tower == True:
                                red_first_tower = True

                        
                    
                    # Inhibs
                    elif timeline_match['info']['frames'][i]['events'][j]['buildingType'] == 'INHIBITOR_BUILDING':
                        lane_type = timeline_match['info']['frames'][i]['events'][j]['laneType']
                        if timeline_match['info']['frames'][i]['events'][j]['teamId'] == 200: # red team inhib destroyed, so we can increase blue team inhib counts
                            if lane_type == "TOP_LANE":
                                blue_top_inhib += 1
                            elif lane_type == "MID_LANE":
                                blue_mid_inhib += 1
                            elif lane_type == "BOT_LANE":
                                blue_bot_inhib += 1
                            else:
                                print("ERROR Inhib")
                        
                        elif timeline_match['info']['frames'][i]['events'][j]['teamId'] == 100: # blue team inhib destroyed, so we can increase red team inhib counts
                            if lane_type == "TOP_LANE":
                                red_top_inhib += 1
                            elif lane_type == "MID_LANE":
                                red_mid_inhib += 1
                            elif lane_type == "BOT_LANE":
                                red_bot_inhib += 1
                            else:
                                print("ERROR Inhib")

                        

                # Elite Monsters, which include dragon, grubs, rift, baron
                elif event_type == 'ELITE_MONSTER_KILL':
                    # Dragons
                    if timeline_match['info']['frames'][i]['events'][j]['monsterType'] == 'DRAGON':
                        dragon_type = timeline_match['info']['frames'][i]['events'][j]['monsterSubType']
                        if timeline_match['info']['frames'][i]['events'][j]['killerTeamId'] == 200: # red team killed the dragon
                            red_air_dragons, red_fire_dragons, red_earth_dragons, red_water_dragons, red_hextech_dragons, red_chemtech_dragons, red_elder_dragons = check_dragon_type(red_air_dragons, red_fire_dragons, red_earth_dragons, red_water_dragons, red_hextech_dragons, red_chemtech_dragons, red_elder_dragons,dragon_type)
                        
                        elif timeline_match['info']['frames'][i]['events'][j]['killerTeamId'] == 100: # blue team killed the dragon
                            blue_air_dragons, blue_fire_dragons, blue_earth_dragons, blue_water_dragons, blue_hextech_dragons, blue_chemtech_dragons, blue_elder_dragons = check_dragon_type(blue_air_dragons, blue_fire_dragons, blue_earth_dragons, blue_water_dragons, blue_hextech_dragons, blue_chemtech_dragons,blue_elder_dragons,dragon_type)

                        

                    
                    # Barons
                    elif timeline_match['info']['frames'][i]['events'][j]['monsterType'] == 'BARON_NASHOR':
                        if timeline_match['info']['frames'][i]['events'][j]['killerTeamId'] == 200: # red team killed the baron
                            red_barons += 1
                        elif timeline_match['info']['frames'][i]['events'][j]['killerTeamId'] == 100: # blue team killed the baron
                            blue_barons += 1

                        

                    # RiftHerald
                    elif timeline_match['info']['frames'][i]['events'][j]['monsterType'] == 'RIFTHERALD':
                        if timeline_match['info']['frames'][i]['events'][j]['killerTeamId'] == 200: # red team killed the rift
                            red_rift = True
                        elif timeline_match['info']['frames'][i]['events'][j]['killerTeamId'] == 100: # blue team killed the rift
                            blue_rift = True
                        
                        

                    # VoidGrubs
                    elif timeline_match['info']['frames'][i]['events'][j]['monsterType'] == 'HORDE':
                        if timeline_match['info']['frames'][i]['events'][j]['killerTeamId'] == 200: # red team killed one horde
                            red_hordes += 1
                        elif timeline_match['info']['frames'][i]['events'][j]['killerTeamId'] == 100: # blue team killed one horde
                            blue_hordes += 1
    


                # Kills, Deaths, and Assists
                elif event_type == 'CHAMPION_KILL':
                    killer_id = timeline_match['info']['frames'][i]['events'][j]['killerId']
                    victim_id = timeline_match['info']['frames'][i]['events'][j]['victimId']
                    try:
                        assistors_list = timeline_match['info']['frames'][i]['events'][j]['assistingParticipantIds']
                    except:
                        assistors_list = []
                        # print("No Assistors in that kill")

                    kills, daths, assists = kills_deaths_assists(killer_id, victim_id, assistors_list,kills,deaths,assists)
                    kda = compute_kda(kills,deaths,assists, kda)      

                # First blood
                elif event_type == "CHAMPION_SPECIAL_KILL" and timeline_match['info']['frames'][i]['events'][j]['killType'] == 'KILL_FIRST_BLOOD':
                    killer_id = timeline_match['info']['frames'][i]['events'][j]['killerId']
                    if killer_id == 0:
                        pass
                    elif killer_id > 0 and killer_id <= 5:  # Blue team killer
                        blue_first_blood = True
                    elif killer_id > 0 and killer_id >= 6:
                        red_first_blood = True


            # Lane minions and Jungle minions
            players_cs_dic = players_cs(timeline_match['info']['frames'][i]['participantFrames'], players_id)

            # Champions Level
            levels = champions_level(timeline_match['info']['frames'][i]['participantFrames'], players_id)

            # Items
            items = collect_items(match_data, players_id)


                            # Add champions level
            for key in list(levels.keys()):
                games_dic.update({
                    key: levels[key]
            })
                
            # Add champions total CS
            for key in list(players_cs_dic.keys()):
                games_dic.update({
                    key: players_cs_dic[key]
                    # round(players_cs_dic[key]/i,2)     # get CS/min rather than CS
                })

            # Add Items
            for key in list(items.keys()):
                games_dic.update({
                    key : items[key]
                })
            

            # Add red and blue turrets
            games_dic.update({"RED Turrets": red_towers,
                            "BLUE Turrets": blue_towers})

            # Add red and blue inhibs
            games_dic.update({"RED TOP INHIB":red_top_inhib,
                                "RED MID INHIB": red_mid_inhib,
                                "RED_BOT_INHIB": red_bot_inhib,
                                "BLUE TOP INHIB": blue_top_inhib,
                                "BLUE MID INHIB": blue_mid_inhib,
                                "BLUE_BOT_INHIB":  blue_bot_inhib})

            # Add red team and blue team killed dragons
            games_dic.update({
                "RED FIRE DRAGON":red_fire_dragons,
                "RED AIR DRAGON": red_air_dragons,
                "RED EARTH DRAGON": red_earth_dragons,
                "RED WATER DRAGON": red_water_dragons,
                "RED HEXTECH DRAGON": red_hextech_dragons,
                "RED CHEMTECH DRAGON": red_chemtech_dragons,
                "RED ELDER DRAGON": red_elder_dragons
            })

            games_dic.update({
                "BLUE FIRE DRAGON":blue_fire_dragons,
                "BLUE AIR DRAGON": blue_air_dragons,
                "BLUE EARTH DRAGON": blue_earth_dragons,
                "BLUE WATER DRAGON": blue_water_dragons,
                "BLUE HEXTECH DRAGON": blue_hextech_dragons,
                "BLUE CHEMTECH DRAGON": blue_chemtech_dragons,
                "BLUE ELDER DRAGON": blue_elder_dragons
            })

            # Add red team and blue team killed barons
            games_dic.update({
                "RED BARON": red_barons,
                "BLUE BARON": blue_barons
            })

            # Add red team and blue team defeated rift
            games_dic.update({
                "RED RIFT": red_rift,
                "BLUE RIFT": blue_rift
            })

            # Add red team and blue team killed VoidGrubs
            games_dic.update({
                "RED GRUBS": red_hordes,
                "BLUE GRUBS": blue_hordes
            })


            # Add each player KDA
            for k in range(10):
                if k <= 4:    # BLUE TEAM
                    games_dic.update({f"BLUE {match_data['info']['participants'][k]['teamPosition']} KDA": kda[k+1]})

                elif k >= 5:
                    games_dic.update({f"RED {match_data['info']['participants'][k]['teamPosition']} KDA": kda[k+1]})


                
            games_dic.update({
                "RED FIRST BLOOD": red_first_blood,
                "BLUE FIRST BLOOD" : blue_first_blood,
                "RED FIRST TOWER": red_first_tower,
                "BLUE FIRST TOWER": blue_first_tower
            })
            

            games_dic.update({
                "TIME STAMP" : 0
            })

            if match_data['info']['teams'][0]['teamId'] == 100: # Blue team
                games_dic.update({
                    "BLUE WIN" : match_data['info']['teams'][0]['win']
                })


               


        games_dic['TIME STAMP'] = timeline_match['info']['frames'][-1]['events'][-1]['timestamp']
        games_dic['MATCH ID'] = timeline_match['metadata']['matchId']

        # Check for region
        if 'EUW' in match:
            games_dic['RANK'] = 'EUW '+ games['tier'][0] + ' ' + games['rank'][0]   
        elif 'NA' in match:
            games_dic['RANK'] = 'NA '+ games['tier'][0] + ' ' + games['rank'][0]  
        elif 'KR' in match:
            games_dic['RANK'] = 'KR ' + games['tier'][0] + ' ' + games['rank'][0] 

        matches_id.append(match)
        return games_dic

    else:
        return False

In [20]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_file_path = os.path.join(parent_dir, 'Puuid\\') 

In [ ]:
# Add the names of the csv file for each region, these files are expceted to be in the Puuid folder
games_KR = pd.read_csv(data_file_path + '', index_col=False)
games_NA = pd.read_csv(data_file_path + '', index_col=False)
games_EUW = pd.read_csv(data_file_path + '', index_col=False)

In [18]:
games_EUW['tier'][0]

'DIAMOND'

In [8]:
KRpuuids = games_KR['Puuid'].to_list()[::-1]
NApuuids = games_NA['Puuid'].to_list()[::-1]
EUWpuuids = games_EUW['Puuid'].to_list()[::-1]

In [9]:
all_data_df_KR = pd.DataFrame()
all_data_df_NA = pd.DataFrame()
all_data_df_EUW = pd.DataFrame()

In [10]:
matches_id = []

In [11]:
matches_id = []

# Specify the file path
file_path = ".txt"

# Open the file and read each line
with open(file_path, "r") as file:
    for line in file:
        # Convert the line to an integer (or float if needed) and append to the list
        matches_id.append(line.strip())


FileNotFoundError: [Errno 2] No such file or directory: 'diamond_matches_id.txt'

In [12]:
for KRpuuid, NApuuid, EUWpuuid in zip(KRpuuids, NApuuids, EUWpuuids):
    # Consider 47 games starting from fifth game
    try:
        print(KRpuuid)
        KR_timeline_matches, response = get_match_ids(KRpuuid, 'asia', dev_key)
        print(KR_timeline_matches)
        NA_timeline_matches, _ = get_match_ids(NApuuid, 'americas', dev_key)    
        print(NA_timeline_matches)
        EUW_timeline_matches, _ = get_match_ids(EUWpuuid, 'europe', dev_key)

        if len(KR_timeline_matches) == 0 or len(NA_timeline_matches) == 0 or len(EUW_timeline_matches) == 0 :
            continue    # Skip invalid puuids

        for KR, NA, EUW in zip(KR_timeline_matches, NA_timeline_matches,EUW_timeline_matches ):
        
            KR_timeline_match = http.request('GET','https://asia.api.riotgames.com/lol/match/v5/matches/'+ KR +'/timeline?api_key='+dev_key).json()
            KR_match_data = http.request('GET','https://asia.api.riotgames.com/lol/match/v5/matches/' + KR +'?api_key='+dev_key).json()

            NA_timeline_match = http.request('GET','https://americas.api.riotgames.com/lol/match/v5/matches/'+ NA +'/timeline?api_key='+dev_key).json()
            NA_match_data = http.request('GET','https://americas.api.riotgames.com/lol/match/v5/matches/' + NA +'?api_key='+dev_key).json()

            EUW_timeline_match = http.request('GET','https://europe.api.riotgames.com/lol/match/v5/matches/'+ EUW +'/timeline?api_key='+dev_key).json()
            EUW_match_data = http.request('GET','https://europe.api.riotgames.com/lol/match/v5/matches/' + EUW +'?api_key='+dev_key).json()


            KR_dic = server_match_data(games_KR,KR, matches_id, KR_match_data,KR_timeline_match)
            NA_dic = server_match_data(games_NA,NA,matches_id , NA_match_data,NA_timeline_match)
            EUW_dic = server_match_data(games_EUW,EUW,matches_id , EUW_match_data,EUW_timeline_match)

            if KR_dic != False:
                all_data_df_KR = pd.concat([all_data_df_KR, pd.DataFrame([KR_dic])], ignore_index=True)

            if NA_dic != False:
                all_data_df_NA = pd.concat([all_data_df_NA, pd.DataFrame([NA_dic])], ignore_index=True)

            if EUW_dic != False:
                all_data_df_EUW = pd.concat([all_data_df_EUW, pd.DataFrame([EUW_dic])], ignore_index=True)

        # Keep track of the dataframe lengthes and indices
        print(f"len of EUW {len(all_data_df_EUW)} | len of NA {len(all_data_df_NA)} | len of KR {len(all_data_df_KR)}")
        print(f"EUW: {EUWpuuids.index(EUWpuuid)}, NA: {NApuuids.index(NApuuid)}, KR: {KRpuuids.index(KRpuuid)}")

        # Remove used Puuid
        games_EUW = games_EUW[games_EUW['Puuid'] != EUWpuuid]
        games_NA = games_NA[games_NA['Puuid'] != NApuuid]
        games_KR = games_KR[games_KR['Puuid'] != KRpuuid]
        
    except:
        if response.status_code == 403:
            break
        print('ERROR', KRpuuid)
        print(response.status_code)
        continue
        
    delay_with_interrupt(122)


F0d2cxhkt0O9zm8svOQnQMn5WC-n69yqtK1RYD1Nal_zAvfYWyff_UKbO1dPDUPGLbH5m8mS3vbOSg
['KR_7444417204', 'KR_7424996634', 'KR_7424559933', 'KR_7423929783', 'KR_7423825413', 'KR_7423764665', 'KR_7423706526', 'KR_7423524731', 'KR_7423382765', 'KR_7420204842', 'KR_7419977588', 'KR_7405604115', 'KR_7405496728', 'KR_7405477231', 'KR_7391028860', 'KR_7383153885', 'KR_7382699688', 'KR_7382587166', 'KR_7382481446', 'KR_7377057789', 'KR_7376620516', 'KR_7376559167', 'KR_7372710467', 'KR_7372646946', 'KR_7372625463', 'KR_7372585390', 'KR_7372519113', 'KR_7371575849', 'KR_7371521427', 'KR_7371426916', 'KR_7371333659', 'KR_7369695865', 'KR_7369652351', 'KR_7365494680', 'KR_7365280207', 'KR_7365171828', 'KR_7365139730', 'KR_7363983537', 'KR_7360753708', 'KR_7360737255', 'KR_7358937400', 'KR_7358903345', 'KR_7357013886', 'KR_7356752080', 'KR_7356669441', 'KR_7356618622', 'KR_7356531835']
[]
7OnU0ZBgzP2kwT3xZ9BmNOwmbKLrHRp9wl6x7eG-51GoQ-p4VioFwK96IKW92czaH-twLANLUn7xFA
['KR_7473630755', 'KR_7473573885', 'KR_

KeyboardInterrupt: 

In [ ]:
# Save the files after removing used puuids
games_KR.to_csv(data_file_path + '')
games_NA.to_csv(data_file_path+ '')
games_EUW.to_csv(data_file_path+ '')

In [ ]:
# Save game data files
all_data_df_EUW.to_csv('EUW_games.csv',index=False)
all_data_df_KR.to_csv('KR_games.csv',index=False)
all_data_df_NA.to_csv('NA_games.csv',index=False)